In [1]:
import numpy as np
import nibabel as nib
import albumentations as A
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, IntSlider, ToggleButtons

__Note: When resizing a mask file make sure it is of type uint8 before resizing to avoid having non-integer outputs for the class labels, after resizing recast to float32 for tensorflow requirements__

### 1- Checking Horizontal flipping

In [2]:
transform = A.Compose(
    p=1.0,
    transforms=[
        A.HorizontalFlip(p=1) # To check if horizontal flip is working
    ]
)

In [3]:
mri = nib.load("../datasets/Task01_BrainTumour/train/images/BRATS_001.nii.gz").get_fdata()
mask = nib.load("../datasets/Task01_BrainTumour/train/masks/BRATS_001.nii.gz").get_fdata().astype(np.uint8)

In [4]:
classes_dict = {
    'Background': 0,
    'Edema': 1,
    'Non-enhancing tumor': 2,
    'Enhancing tumor': 3 
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Edema', 'Non-enhancing tumor', 'Enhancing tumor', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=154, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer, channel):
    print(f"Plotting Layer: {layer} | Label: {seg_class} | Channel: {channel}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.title("MRI", fontsize=20)
    plt.imshow(mri[:, :, layer, channel], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    if seg_class == "All":
        mask_img = mask[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask_img)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask_img = np.where(mask[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask_img, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: "FLAIR" Fluid-attenuated inversion recovery
#  Channel 1: "T1w" T1-weighted
#  Channel 2: "t1gd" T1-weighted with gadolinium contrast enhancement
#  Channel 3: "T2w" T2-weighted    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer, channel=(0, 3))

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

In [5]:
transformed = transform(image=mri, mask=mask)
mri = transformed["image"]
mask = transformed["mask"]

In [6]:
mri.dtype

dtype('float64')

In [7]:
mri.shape

(240, 240, 155, 4)

In [8]:
mask.dtype

dtype('uint8')

In [9]:
mask.shape

(240, 240, 155)

In [10]:
classes_dict = {
    'Background': 0,
    'Edema': 1,
    'Non-enhancing tumor': 2,
    'Enhancing tumor': 3 
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Edema', 'Non-enhancing tumor', 'Enhancing tumor', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=154, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer, channel):
    print(f"Plotting Layer: {layer} | Label: {seg_class} | Channel: {channel}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.title("MRI", fontsize=20)
    plt.imshow(mri[:, :, layer, channel], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    if seg_class == "All":
        mask_img = mask[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask_img)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask_img = np.where(mask[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask_img, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: "FLAIR" Fluid-attenuated inversion recovery
#  Channel 1: "T1w" T1-weighted
#  Channel 2: "t1gd" T1-weighted with gadolinium contrast enhancement
#  Channel 3: "T2w" T2-weighted    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer, channel=(0, 3))

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### 2- Checking Resizing from size 400x400x180 to 320x320x180

In [11]:
transform = A.Compose(
    p=1.0,
    transforms=[
        A.Resize(
            height=320,
            width=320,
            interpolation=1,
            p=1
        ) # To check if Resizing is working
    ]
)

In [12]:
mri = nib.load("../datasets/Task02_Heart/test_images_for_model_prediction_submission/la_006.nii.gz").get_fdata()

In [13]:
mri.shape

(400, 400, 180)

In [14]:
transformed = transform(image=mri)

In [15]:
resized_mri = transformed["image"]

In [16]:
resized_mri.shape

(320, 320, 180)

In [17]:
# Create layer slider
select_layer = IntSlider(min=0, max=mri.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(layer):
    print(f"Plotting Layer: {layer}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.title("MRI", fontsize=20)
    plt.imshow(mri[:, :, layer], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    plt.title("Resized MRI", fontsize=20)
    plt.imshow(resized_mri[:, :, layer], cmap='gray')
    plt.axis('off');
    
# Use the interactive() tool to create the visualization
interactive(plot, layer=select_layer)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Layer', max=179), Output…

### 3- Checking Resizing from 256x256x18x2 to 320x320x18x2
__Note: it seems for resizing each channel must be resized separately, having both channels would not work__

In [18]:
transform = A.Compose(
    p=1.0,
    transforms=[
        A.Resize(
            height=320,
            width=320,
            interpolation=1,
            p=1
        ) # To check if Resizing is working
    ]
)

In [19]:
mri = nib.load("../datasets/Task05_Prostate/train/images/prostate_18.nii.gz").get_fdata().astype(np.float32)

In [20]:
mri.shape

(256, 256, 18, 2)

In [21]:
# IMPORTANT TO CAST AS TYPE UINT8 AND THEN RECASTING TO TYPE FLOAT32 AFTER RESIZING TO AVOID NON-INTEGER VALUES AFTER RESIZING
mask = nib.load("../datasets/Task05_Prostate/train/masks/prostate_18.nii.gz").get_fdata().astype(np.uint8)

In [22]:
mask.shape

(256, 256, 18)

In [23]:
np.unique(mask)

array([0, 1], dtype=uint8)

In [24]:
transformed_channel1 = transform(image=mri[:, :, :, 0])
transformed_channel2 = transform(image=mri[:, :, :, 1])
transformed_mask = transform(image=mask)

In [25]:
resized_mri_channel1 = transformed_channel1["image"]
resized_mri_channel2 = transformed_channel2["image"]
resized_mask = transformed_mask["image"]

In [26]:
resized_mri_channel1.shape

(320, 320, 18)

In [27]:
resized_mri_channel2.shape

(320, 320, 18)

In [28]:
resized_mri = np.stack((resized_mri_channel1, resized_mri_channel2), axis = -1)

In [29]:
resized_mri.shape

(320, 320, 18, 2)

In [30]:
resized_mask.shape

(320, 320, 18)

In [31]:
np.unique(resized_mask)

array([0, 1], dtype=uint8)

In [32]:
# For python variable assignment issues
orig_mri = mri
orig_mask = mask

classes_dict = {
    'Background': 0,
    'Peripheral-Zone': 1,
    'Transition-Zone': 2
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Peripheral-Zone', 'Transition-Zone', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=orig_mri.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer, channel):
    print(f"Plotting Layer: {layer} | Label: {seg_class} | Channel: {channel}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(2, 2, 1)
    plt.imshow(orig_mri[:, :, layer, channel], cmap='gray');
    plt.title("MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(2, 2, 2)
    if seg_class == "All":
        mask = orig_mask[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(orig_mask[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');
    
    fig.add_subplot(2, 2, 3)
    plt.imshow(resized_mri[:, :, layer, channel], cmap='gray');
    plt.title("Resized MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(2, 2, 4)
    if seg_class == "All":
        mask = resized_mask[:, :, layer]
        plt.title("Resized Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(resized_mask[:, :, layer] == img_label, 255, 0)
        plt.title("Resized Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: T2
#  Channel 1: ADC    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer, channel=(0, 1))

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### 4- Checking Resizing from size 35x51x35 to 32x32x35

In [64]:
transform = A.Compose(
    p=1.0,
    transforms=[
        A.Resize(
            height=32,
            width=32,
            interpolation=1,
            p=1
        ) # To check if Resizing is working
    ]
)

In [65]:
mri = nib.load("../datasets/Task04_Hippocampus/train/images/hippocampus_001.nii.gz").get_fdata()

In [66]:
mri.shape

(35, 51, 35)

In [67]:
# IMPORTANT TO CAST AS TYPE UINT8 AND THEN RECASTING TO TYPE FLOAT32 AFTER RESIZING TO AVOID NON-INTEGER VALUES AFTER RESIZING
mask = nib.load("../datasets/Task04_Hippocampus/train/masks/hippocampus_001.nii.gz").get_fdata().astype(np.uint8)

In [68]:
mask.shape

(35, 51, 35)

In [69]:
np.unique(mask)

array([0, 1, 2], dtype=uint8)

In [70]:
transformed = transform(image=mri, mask=mask)

In [71]:
resized_mri = transformed["image"]
resized_mask = transformed["mask"].astype(np.uint8)

In [72]:
resized_mri.shape

(32, 32, 35)

In [73]:
resized_mask.shape

(32, 32, 35)

In [74]:
np.unique(resized_mask)

array([0, 1, 2], dtype=uint8)

In [75]:
# For python variable assignment issues
orig_mri = mri
orig_mask = mask

classes_dict = {
    'Background': 0,
    'Anterior': 1,
    'Exterior': 2
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Anterior', 'Exterior', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=orig_mri.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer):
    print(f"Plotting Layer: {layer} | Label: {seg_class}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(2, 2, 1)
    plt.imshow(orig_mri[:, :, layer], cmap='gray');
    plt.title("MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(2, 2, 2)
    if seg_class == "All":
        mask = orig_mask[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(orig_mask[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');
    
    fig.add_subplot(2, 2, 3)
    plt.imshow(resized_mri[:, :, layer], cmap='gray');
    plt.title("Resized MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(2, 2, 4)
    if seg_class == "All":
        mask = resized_mask[:, :, layer]
        plt.title("Resized Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(resized_mask[:, :, layer] == img_label, 255, 0)
        plt.title("Resized Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: T2
#  Channel 1: ADC    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer)

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### 5- Checking reverse resizing from size 32x32x35 to 35x51x35 

In [45]:
transform = A.Compose(
    p=1.0,
    transforms=[
        A.Resize(
            height=35,
            width=51,
            interpolation=1,
            p=1
        ) # To check if Resizing is working
    ]
)

In [46]:
transformed_mri = transform(image=mri)
transformed_mask = transform(image=mask)

In [47]:
resized_mri = transformed_mri["image"]
resized_mask = transformed_mask["image"]

In [48]:
resized_mri.shape

(35, 51, 35)

In [49]:
resized_mask.shape

(35, 51, 35)

In [50]:
np.unique(resized_mask)

array([0, 1, 2], dtype=uint8)

In [51]:
# For python variable assignment issues
orig_mri = mri
orig_mask = mask

classes_dict = {
    'Background': 0,
    'Anterior': 1,
    'Exterior': 2
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Anterior', 'Exterior', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=orig_mri.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer):
    print(f"Plotting Layer: {layer} | Label: {seg_class}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(2, 2, 1)
    plt.imshow(orig_mri[:, :, layer], cmap='gray');
    plt.title("MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(2, 2, 2)
    if seg_class == "All":
        mask = orig_mask[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(orig_mask[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');
    
    fig.add_subplot(2, 2, 3)
    plt.imshow(resized_mri[:, :, layer], cmap='gray');
    plt.title("Resized MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(2, 2, 4)
    if seg_class == "All":
        mask = resized_mask[:, :, layer]
        plt.title("Resized Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(resized_mask[:, :, layer] == img_label, 255, 0)
        plt.title("Resized Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: T2
#  Channel 1: ADC    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer)

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### 6- Checking Resizing from size 512x512x75 to 320x320x75

In [52]:
transform = A.Compose(
    p=1.0,
    transforms=[
        A.Resize(
            height=320,
            width=320,
            interpolation=1,
            p=1
        ) # To check if Resizing is working
    ]
)

In [53]:
mri = nib.load("../datasets/Task03_Liver/train/images/liver_0.nii.gz").get_fdata()

In [54]:
mri.shape

(512, 512, 75)

In [55]:
mask = nib.load("../datasets/Task03_Liver/train/masks/liver_0.nii.gz").get_fdata()

In [56]:
mask.shape

(512, 512, 75)

In [57]:
transformed = transform(image=mri)

In [58]:
resized_mri = transformed["image"]

In [59]:
resized_mri.shape

(320, 320, 75)

In [60]:
transformed = transform(image=mask.astype(np.uint8))

In [61]:
resized_mask = transformed["image"].astype(np.uint8)

In [62]:
resized_mask.shape

(320, 320, 75)

In [63]:
# For python variable assignment issues
orig_mri = mri
orig_mask = mask

classes_dict = {
    'Background': 0,
    'Liver': 1,
    'Cancer': 2
}

# Create button values
select_class = ToggleButtons(
    options=['Background', 'Liver', 'Cancer', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=orig_mri.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer):
    print(f"Plotting Layer: {layer} | Label: {seg_class}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(2, 2, 1)
    plt.imshow(orig_mri[:, :, layer], cmap='gray');
    plt.title("MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(2, 2, 2)
    if seg_class == "All":
        mask = orig_mask[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(orig_mask[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');
    
    fig.add_subplot(2, 2, 3)
    plt.imshow(resized_mri[:, :, layer], cmap='gray');
    plt.title("Resized MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(2, 2, 4)
    if seg_class == "All":
        mask = resized_mask[:, :, layer]
        plt.title("Resized Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(resized_mask[:, :, layer] == img_label, 255, 0)
        plt.title("Resized Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: T2
#  Channel 1: ADC    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer)

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…